In [391]:
from sbdataextraction import sbdataextraction as sb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import uniform

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split, KFold
from sklearn.metrics import log_loss

import pyjags as pjs

In [4]:
la_liga = sb.fetch_seasons_for_league(11)

Matches will be fetched for 15 seasons.

Fetching matches for season_id 4 of competition_id 11...
[=================================] 100%
Fetching matches for season_id 1 of competition_id 11...
[===================================] 100%
Fetching matches for season_id 2 of competition_id 11...
[=================================] 100%
Fetching matches for season_id 27 of competition_id 11...
[================================] 100%
Fetching matches for season_id 26 of competition_id 11...
[=====================================] 100%
Fetching matches for season_id 25 of competition_id 11...
[==============================] 100%
Fetching matches for season_id 24 of competition_id 11...
[===============================] 100%
Fetching matches for season_id 23 of competition_id 11...
[====================================] 100%
Fetching matches for season_id 22 of competition_id 11...
[================================] 100%
Fetching matches for season_id 21 of competition_id 11...
[==========

In [9]:
la_liga_shots = sb.get_shots_for_league(la_liga)

Getting shots for 2018/2019
Getting shots for 2017/2018
Getting shots for 2016/2017
Getting shots for 2015/2016
Getting shots for 2014/2015
Getting shots for 2013/2014
Getting shots for 2012/2013
Getting shots for 2011/2012
Getting shots for 2010/2011
Getting shots for 2009/2010
Getting shots for 2008/2009
Getting shots for 2007/2008
Getting shots for 2006/2007
Getting shots for 2005/2006
Getting shots for 2004/2005
Done.


In [12]:
la_liga_shots["Distance"] = np.sqrt((la_liga_shots["x start location"] - 120)**2 + \
                                    (la_liga_shots["y start location"] - 40)**2)
goal_label_mask = la_liga_shots["outcome"].str.contains("Goal")
la_liga_shots["outcome"].loc[goal_label_mask] = 1
la_liga_shots["outcome"].loc[~goal_label_mask] = 0

/Users/robertblumberg/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [47]:
la_liga_shots.head(5)

,team_id,team_name,player_id,player_name,play pattern,x start location,y start location,duration,outcome,technique,first time,x gk position,y gk position,type of shot,num opponents within 5 yards,num opponents between shot and goal,statsbomb xg,game_id,season_id,Distance
shot id,,,,,,,,,,,,,,,,,,,,
6c30cac4-6f3b-4cd0-9913-26137b058416,218,Real Betis,6658,Cristian Tello Herrera,From Throw In,101.0,22.0,1.179654,0,Normal,False,118.6,37.9,Open Play,2,2,0.020549,16215,2018/2019,26.172505
67cfa552-952c-4f72-a33a-0dd6af217a66,218,Real Betis,3990,Jesé Rodríguez Ruiz,From Throw In,113.7,39.9,1.226969,0,Volley,True,117.2,39.1,Open Play,1,0,0.363959,16215,2018/2019,6.300794
8c95e369-ec54-4a84-9d3e-51abb6f38e6e,217,Barcelona,5503,Lionel Andrés Messi Cuccittini,From Free Kick,95.6,44.4,1.046942,1,Normal,False,119.5,39.0,Free Kick,0,5,0.127337,16215,2018/2019,24.793548
2ae506b9-0601-460b-b91d-802296120161,217,Barcelona,5503,Lionel Andrés Messi Cuccittini,From Free Kick,94.0,46.1,1.375915,0,Normal,False,119.1,39.5,Free Kick,0,4,0.090603,16215,2018/2019,26.705992
10dc9818-a8d2-47f9-b219-9bc87d7b22e6,218,Real Betis,6658,Cristian Tello Herrera,From Free Kick,104.1,22.2,0.116586,0,Normal,False,118.4,37.4,Open Play,3,1,0.022751,16215,2018/2019,23.867342


In [405]:
features= la_liga_shots.drop(["outcome"], axis=1)
targets= la_liga_shots["outcome"]

X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2)
X_train, X_valid, y_train, y_valid = train_test_split(features.loc[X_train.index], targets[X_train.index], test_size=0.2)

shot_training_set = pd.merge(X_train, y_train, left_index=True, right_index=True)
shot_valid_set = pd.merge(X_valid, y_valid, left_index=True, right_index=True)
shot_test_set = pd.merge(X_test, y_test, left_index=True, right_index=True)

In [406]:
class AttrSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, attributes):
        self.attributes = attributes
   
    def fit(self, attributes):
        return self
    
    def transform(self, X):
        return X[self.attributes]

def full_preprocessing_pipeline(X_train, X_test, cat_features, num_features, fit=True):
    """
    """
    
    cat_attr_pipeline = Pipeline([
        ('selector', AttrSelector(cat_features)),
        ('one_hot_encoder',  OneHotEncoder())
    ])
    
    num_attr_pipeline = Pipeline([
        ('selector', AttrSelector(num_features)),
       #('poly', PolynomialFeatures(2)), 
        ('std_scaler', StandardScaler())
    ])
    
    combined_pipeline = ColumnTransformer(transformers=[('num', num_attr_pipeline, num_features),
                                                        ('cat', cat_attr_pipeline, cat_features)])
    
    if fit:
        X_train_preprocessed = combined_pipeline.fit_transform(X_train)
        X_test_preprocessed = combined_pipeline.transform(X_test)
    
        return X_train_preprocessed, X_test_preprocessed, combined_pipeline
    
    return combined_pipeline

In [407]:
cat_shot_attributes = ["play pattern", "technique", "first time"]
num_shot_attributes = ["x start location", "y start location", "x gk position", 
                       "y gk position", "num opponents within 5 yards", 
                       "num opponents between shot and goal", "Distance"]

X_train_prep, X_valid_prep, preprocessor = full_preprocessing_pipeline(X_train=shot_training_set,
                                                                       X_test=shot_valid_set,
                                                                       cat_features=cat_shot_attributes,
                                                                       num_features=num_shot_attributes,
                                                                       fit=True)

In [408]:
cat_features_transformed = preprocessor.transformers[1][1]['one_hot_encoder'] \
                                       .fit(X_train[cat_shot_attributes], y_train) \
                                       .get_feature_names(cat_shot_attributes)

all_features_transformed = list(num_shot_attributes) + list(cat_features_transformed)

In [409]:
def hyperparameter_opt(classifier, hyprparam_dist, X_train, y_train, scoring="f1", n_iter=10):
    """
    Runs randomized hyperparameter search on preprocessed data for specified classifer.
        
    Arguments:
    ----------
    classifier : sklearn classifier object
        - classifer we want to optimize
    hyprparam_dist : dict or list of dicts
        - dictionary with parameters names (string) as keys and distributions or 
        lists of hyperparameters to try
    X_train : pandas.DataFrame
        - preprocessed training features
    y_train : pandas.Series
        - training labels
            
    Returns:
    --------
    (pandas.DataFrame, dict)
        - hyperparameter cross validation results, and dict with best hyperparameters
    """ 

    print("Optimizing hyperparameters for {} model...".format(type(classifier).__name__))
    rs = RandomizedSearchCV(estimator=classifier, 
                            param_distributions=hyprparam_dist, 
                            cv=10, verbose=0, 
                            scoring=scoring, n_iter=n_iter)
    
    rs.fit(X_train, y_train)
    print("Best hyperparameters {0}".format(rs.best_params_))
    print("Best cross validation score {0}\n".format(rs.best_score_))

    return pd.DataFrame(rs.cv_results_), rs.best_estimator_

In [410]:
lr = LogisticRegression(max_iter=10000)
cv_results, fitted_lr = hyperparameter_opt(classifier=lr, 
                                           hyprparam_dist=[{"C" : uniform(0, 20000)}], 
                                           X_train=X_train_prep, 
                                           y_train=y_train.astype(int),
                                           scoring="neg_log_loss",
                                           n_iter=50)

Optimizing hyperparameters for LogisticRegression model...
Best hyperparameters {'C': 1590.4930144198404}
Best cross validation score -0.3214776101189263



In [411]:
fitted_lr.coef_[0]

array([-0.87693225,  0.10832701, -0.40143425, -0.01439687, -0.26892546,
       -0.09487879, -2.03568582, -0.80546895, -0.09148253, -0.20838915,
       -0.37987539, -0.97980222, -0.06840543, -0.41536553,  1.83747157,
       -0.2600266 , -1.12872396,  0.20040483, -0.35058354,  0.52050658,
        0.11572493, -0.44900569, -0.27966739, -0.91287116, -0.45847307])

In [432]:
log_loss(shot_valid_set["outcome"].astype(int), fitted_lr.predict_proba(X_valid_prep)[:, 1])

0.3312087909375898

In [412]:
lr_coefs = np.append(fitted_lr.intercept_, fitted_lr.coef_[0])

In [413]:
training_set_player = shot_training_set.set_index(["player_id"]).sort_index()
valid_set_player = shot_valid_set.set_index(["player_id"]).sort_index()
X_train_player_prep, X_valid_player_prep, prprocesr = full_preprocessing_pipeline(X_train=training_set_player,
                                                                                  X_test=valid_set_player,
                                                                                  cat_features=cat_shot_attributes,
                                                                                  num_features=num_shot_attributes,
                                                                                  fit=True)
nrow_t = X_train_player_prep.shape[0]
nrow_v = X_valid_player_prep.shape[0]
X_train_player_prep = np.append(np.ones((nrow_t, 1)), X_train_player_prep, axis=1)
X_valid_player_prep = np.append(np.ones((nrow_v, 1)), X_valid_player_prep, axis=1)

In [414]:
np.array(training_set_player["outcome"])

array([0, 0, 0, ..., 0, 0, 0], dtype=object)

In [415]:
training_set_player.index.unique().shape[0]

932

In [416]:
X_train_player_prep

array([[ 1.        , -1.30123075,  0.77503826, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.        , -0.90390136,  0.05266482, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.        , -1.30123075, -1.7780076 , ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 1.        ,  1.03859786, -0.20461887, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.        , -1.2018984 , -1.6691568 , ...,  0.        ,
         1.        ,  0.        ],
       [ 1.        ,  0.2880868 , -1.58999259, ...,  0.        ,
         1.        ,  0.        ]])

In [417]:
jags_code = ''' 
model {
  for (k in 1:M) {
    a[k] ~ dnorm(coef[k], 1/0.001)
  }
  
  for (i in 1:n_players) {
    count[i] = sum(nshot_by_player[1:i])-nshot_by_player[i]
    for (shot in 1:nshot_by_player[i]) {
      p[sum(nshot_by_player[1:i])-nshot_by_player[i] + shot] = 1 / (1 + exp(-sum(a*X[sum(nshot_by_player[1:i])-nshot_by_player[i] + shot, ])))
      y[sum(nshot_by_player[1:i])-nshot_by_player[i] + shot] ~ dbern(p[sum(nshot_by_player[1:i])-nshot_by_player[i] + shot])
    }
  }
}'''

In [418]:
jags_data = {
    "M" : lr_coefs.shape[0],
    "coef" : lr_coefs,
    "n_players" : shot_counts_by_player.shape[0],
    "nshot_by_player" : shot_counts_by_player.tolist(),
    "y" : training_set_player["outcome"].astype(int).tolist(),
    "X" : X_train_player_prep
}

In [419]:
model = pjs.Model(code = jags_code, data = jags_data, chains = 1, adapt=100) 

adapting: iterations 5 of 100, elapsed 0:00:05, remaining 0:01:42
adapting: iterations 9 of 100, elapsed 0:00:11, remaining 0:01:49
adapting: iterations 17 of 100, elapsed 0:00:20, remaining 0:01:37
adapting: iterations 25 of 100, elapsed 0:00:29, remaining 0:01:27
adapting: iterations 33 of 100, elapsed 0:00:38, remaining 0:01:17
adapting: iterations 41 of 100, elapsed 0:00:47, remaining 0:01:07
adapting: iterations 49 of 100, elapsed 0:00:56, remaining 0:00:58
adapting: iterations 57 of 100, elapsed 0:01:04, remaining 0:00:48
adapting: iterations 65 of 100, elapsed 0:01:12, remaining 0:00:39
adapting: iterations 73 of 100, elapsed 0:01:19, remaining 0:00:29
adapting: iterations 81 of 100, elapsed 0:01:27, remaining 0:00:20
adapting: iterations 89 of 100, elapsed 0:01:35, remaining 0:00:12
adapting: iterations 97 of 100, elapsed 0:01:43, remaining 0:00:03
adapting: iterations 100 of 100, elapsed 0:01:46, remaining 0:00:00


In [420]:
samples = model.sample(1, vars=['p'], thin=1)

sampling: iterations 1 of 1, elapsed 0:00:01, remaining 0:00:00


In [421]:
samples['p'].flatten()

array([0.01715234, 0.03003776, 0.01063729, ..., 0.17740681, 0.00739781,
       0.0304871 ])

In [422]:
log_loss(training_set_player["outcome"].astype(int), samples['p'].flatten())

0.3170332159168761

In [423]:
log_loss(training_set_player["outcome"].astype(int), training_set_player["statsbomb xg"])

0.30005081492228775